In [6]:
# COPYRIGHT  : https://apmonitor.com/me575/index.php/Main/ScheduleOptimization

""" 
Objectif : Minimiser le délai pour la production de 5 produits sur 2 machines. 

Chaque produit nécessite un temps de traitement différent pour chaque étape.

Il existe 5 produits qui nécessitent 2 étapes de traitement sur des machines distinctes. 

Les deux machines (machine Y et machine Z) fonctionnent l'une après l'autre. 

Un produit doit d'abord passer par la machine Y et ensuite par la machine Z. 

Les deux machines peuvent fonctionner simultanément, mais chaque machine ne peut pas travailler sur plus 

d'un produit à la fois. 

L'usine doit minimiser le temps de production de tous les produits donnés. 

Cela revient à minimiser le temps d'inactivité de la machine Z.


 """
from IPython.display import Image
Image(url= "1.png", width=400, height=200)

# Ci - dessous, on voit que contruire Le produit P0 mets 10 minutes sur la machine Y 
# puis 20 minutes sur la machine Z.


In [7]:
# Option 1 : Une recherche exhaustive

# Il y a n! permutations pour une liste de production de longueur n. 
# Pour ce problème spécifique avec 5 produits, il y a 5!=120 combinaisons. 
# Une recherche exhaustive calcule le makepan (temps total) pour toutes les combinaisons possibles.

from itertools import permutations
import pandas as pd

Y = {0: 10, 1: 20, 2: 15, 3: 40, 4: 8}
Z = {0: 20, 1: 30, 2: 10, 3: 25, 4: 18}

products = Y.keys()
np = len(products)
pm = [] # list of permutations
tm = [] # list of makespan time
for order in permutations(products):
    # append permutation to the pm list
    pm.append(order)
    # lookup time for each step
    ty = [0]*(np+1); tz = [0]*(np+1)
    for i in range(np):
        ty[i] = Y[order[i]]
        tz[i+1] = Z[order[i]]
    # calculate makespan time
    t = 0
    for i in range(np+1):
        t += max(ty[i],tz[i])
    tm.append(t)

# convert to Pandas DataFrame for sorting
r = pd.DataFrame(pm,columns=['T0','T1','T2','T3','T4'])
r['makespan'] = tm
r.sort_values('makespan',inplace=True)
print(r.head(10))

#Les 10 meilleurs horaires montrent qu'il existe deux meilleurs horaires candidats, chacun avec un intervalle de 121.


     T0  T1  T2  T3  T4  makespan
97    4   0   1   3   2       121
108   4   2   0   1   3       121
2     0   1   3   2   4       123
3     0   1   3   4   2       123
106   4   1   3   0   2       123
107   4   1   3   2   0       123
10    0   2   4   1   3       125
19    0   4   1   3   2       125
66    2   4   0   1   3       128
48    2   0   1   3   4       128


In [ ]:
# Dans le tableau précédent, Sur la ligne 1, le 4 veut dire 'P4' , puis 'P0' , puis 'P1',  puis 'P3', puis 'P2'
# en fait c'est l'ordre de production des produits, afin de réduire la durée finale d'exécution ( makespan).
# On ne comprends pas de suite car on ne retrouve pas la lettre P

In [ ]:
# Bien qu'une recherche exhaustive soit rapide pour ce problème, 
# il devient impossible de calculer toutes les combinaisons pour des horaires plus complexes. 
# Les méthodes heuristiques ou d'optimisation sont utilisées lorsque la recherche exhaustive 
# est trop exigeante en termes de calcul.

In [8]:
# Option 2 : Utiliser l'algorithme'"Johnson Rule"
# https://en.wikipedia.org/wiki/Johnson%27s_rule

""" La technique nécessite plusieurs conditions préalables :

    Le temps pour chaque tâche doit être constant.
    Les temps de travail doivent être mutuellement exclusifs de la séquence de travail.
    Tous les travaux doivent être traités dans le premier centre de travail avant de passer par le deuxième centre de travail.
    Tous les emplois sont également prioritaires.

La règle de Johnson est la suivante :

    Dressez la liste des emplois et de leurs temps à chaque centre de travail.
    Sélectionnez le travail avec le temps d'activité le plus court. Si ce temps d'activité est pour le premier poste de travail, planifiez d'abord le travail. Si ce temps d'activité est pour le deuxième poste de travail, planifiez le travail en dernier. Rompre les liens arbitrairement.
    Éliminer le travail le plus court de toute considération ultérieure.
    Répétez les étapes 2 et 3, en travaillant vers le centre de la planification des tâches jusqu'à ce que toutes les tâches aient été planifiées. """


# Semblable à l'optimisation du sac à dos, 
# il existe une solution heuristique spécifique au problème et qui permet d'obtenir 
# des résultats optimaux sous certaines hypothèses restrictives. 
# La règle de Johnson est une méthode de planification des travaux dans deux postes de travail avec un temps 
# de travail constant. Les temps de travail ne sont pas affectés par la séquence et les produits sont traités 
# sur la première machine avant de passer à la deuxième machine. La règle de Johnson sélectionne le produit avec 
# le temps d'activité le plus court et l'affecte à être traité en premier si le temps le plus court est pour 
# la machine Y. Si le temps le plus court est pour la machine Z, il affecte le produit à la fin de la liste. 
# Le temps le plus court suivant est sélectionné et attribué comme deuxième ou avant-dernier produit 
# si ceux-ci ont été sélectionnés précédemment. Le processus est répété jusqu'à ce que tous les produits 
# soient commandés.

# Le temps le plus bas est P4 et il est attribué en premier car le temps le plus bas est pour la machine Y.
# Ensuite, le temps le plus bas suivant est P2 et il est affecté en dernier 
# car le temps le plus bas est pour la machine Z.

# La commande est P4, P0, P1, P3, P2 avec un retard total de 26 sur la machine Z 
# et un temps total de fabrication de 121 pour terminer tout le traitement. 
# Il s'agit de la même ligne que la première ligne de la liste de recherche exhaustive triée.

Image(url= "2.png", width=400, height=200)


In [ ]:
# Option 3 : Programmation en nombres entiers
# Les mêmes heuristiques ne fonctionnent pas s'il y a des changements dans les hypothèses qui 
# invalideraient la règle de Johnson. Cela peut se produire si le problème change 
# ou s'il existe une priorité pour la finition d'un produit par rapport à un autre.
#  Il peut également y avoir des contraintes supplémentaires, une troisième étape de traitement pourrait 
#  être introduite. Une approche plus flexible consiste à résoudre le problème avec la programmation 
#  linéaire entière (ILP). Des variables, des équations et un objectif sont créés pour tenir dans 
#  un formulaire standard qu'un optimiseur peut résoudre.



